# Useful Code Snippets

## Databases

### Connect via ODBC

In [ ]:
import pyodbc
import getpass

OL_dsn = 'Olive_Prod'  # My DSN set up in ODBC manager 
OL_uid = 'thompsonja'
OL_pwd = 'xxxxxx' #getpass.getpass('Enter Olive Password: ')
OL_LOGIN = 'DSN='+ OL_dsn +';UID='+ OL_uid +';PWD=' + OL_pwd 
OL_conn = pyodbc.connect(OL_LOGIN, autocommit=True) 

### Run SQL

In [ ]:
mycursor = OL_conn.cursor()

sql = """
......
"""

mycursor.execute(sql)
mycursor.fetchall() # to fetch all results - not required for Create table etc
mycursor.commit()

In [ ]:
    def get_config_file(self):
        """
        To avoid typing the password in everytime when logging into
        Olive or Netezza, these are saved in a config file. Read this in.
        """        
        self.db_config = {}
        with open(self.db_config_loc) as f:
            for line in f:
               (key, val) = line.split()
               self.db_config[key] = val


    def olive_login(self):
        """
        Login to Olive
        """
        OL_dsn = 'Olive_Prod'  # My DSN set up in ODBC manager 
        OL_uid = 'thompsonja'
        OL_pwd = self.db_config['Olive']
        OL_LOGIN = 'DSN='+ OL_dsn +';UID='+ OL_uid +';PWD=' + OL_pwd 
        self.OL_conn = pyodbc.connect(OL_LOGIN, autocommit=True) 
        print(self.OL_conn.getinfo(pyodbc.SQL_DRIVER_NAME))

### Insert values into table: helper function

In [ ]:
def loadValuesIntoTable(table_name, values_dic):
    """
    Function to create a SQL string 'insert into table_name values (val1, val2, etc)' that can be executed.
    table_name: the name of the table to insert the data into
    values_dic: keys are the values to be inserted, if the values are True then treated as a string (i.e. '' added) 
                otherwise treated as a number
    
    Returns an sql string that can be executed
    """
    values_str = ''
    for k, v in values_dic.items():        
        if v:
            # string
            value = """'""" + k + """', """
        else:
            # number
            value = str(k) + ', '
        values_str += value
    
    values_str = values_str[:-2]
    
    sql = """ insert into {table_name} values ({values_str});""".format(table_name=table_name, values_str=values_str)  
    return sql

## Outlook

In [ ]:
import win32com.client as win32

### Send email

In [ ]:
outlook = win32.Dispatch('outlook.application')
mail = outlook.CreateItem(0)
mail.To = 'jason.thompson@sky.uk; jonathan.green2@sky.uk'
mail.Subject = 'Panel Expansion: 250k Data Ready'
mail.Body = ''
#mail.HTMLBody = '<h2>HTML Message body</h2>' #this field is optional

# To attach a file to the email (optional):
#attachment  = "Path to the attachment"
#mail.Attachments.Add(attachment)

mail.Send()

### Search emails

In [ ]:
# return all messages in subfldr_index which is a subfolder of inbox_index
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.GetDefaultFolder(6)
subfldr = inbox.Folders[11]
messages = subfldr.Items
# identify most recent message
dt = messages[0].ReceivedTime
for message in messages:
    if message.ReceivedTime >= dt:
        dt = message.ReceivedTime
        subject = message.Subject

# check the most recent email
if 'ENABLED' in subject:
    return True
else:
    return False

## Summarise the values in a dataframe

In [ ]:
def table_value_analysis(table_df):
    """
    Go through each column of table_df and summarise:
        1. the number of unique values
        2. the value with the highest count
        3. the count of this value
        4. the % of rows with this value

    Return resuts as a dataframe
    """
    import pandas as pd


    column_names = list(table_df)
    column_summaries = []        

    for i in range(table_df.shape[1]):
        num_of_unique_values = table_df.iloc[:, i].nunique()
        if num_of_unique_values > 0:
            count_values = pd.DataFrame(table_df.groupby(column_names[i])[column_names[i]].count()).rename(mapper={column_names[i] : 'count'}, axis=1)
            sort_values = count_values.sort_values(['count'], ascending=False)
            column_summary = [column_names[i], num_of_unique_values, sort_values.index.values[0], sort_values.iloc[0, 0]]
            column_summaries.append(column_summary)
        else:
            column_summary = [column_names[i], num_of_unique_values, -1, -1]


    column_summaries_df = pd.DataFrame(column_summaries, columns=['Column Name', 'Unique Values', 
                                                                  'Value With Highest Count', 'Highest Count'])

    column_summaries_df['Highest %'] = 100 * column_summaries_df['Highest Count'] / table_df.shape[0]
    column_summaries_df = column_summaries_df.sort_values(['Highest %'], ascending=False)        

    return column_summaries_df        

## BigQuery

In [ ]:
def bq_create_table(sql, client, dataset, 
                    table_name, truncate=True):
    """
    Create a table in BigQuery from the result of the SQL
    """
    from google.cloud import bigquery

    # set up table creation details to hold query results
    create_instruction = bigquery.job.CreateDisposition.CREATE_IF_NEEDED
    if truncate:            
        write_instruction = bigquery.job.WriteDisposition.WRITE_TRUNCATE
    else:
        write_instruction = bigquery.job.WriteDisposition.WRITE_APPEND
    job_config = bigquery.QueryJobConfig()
    job_config.create_disposition = create_instruction
    job_config.write_disposition = write_instruction
    job_config.destination = dataset.table(table_name)

    # Run the Query 
    query_job = client.query(sql, job_config=job_config)
    # execute the query
    iterator = query_job.result()  # Wait for job to complete
    job_result = query_job.state         

## Get Name of All Files in a Folder

In [ ]:
import os
from os import walk

os.chdir(barb_dir)    

all_files = []    
for (dirpath, dirnames, filenames) in walk('.'):
    all_files.extend(filenames)
    break   

## Dates

In [1]:
import datetime
from datetime import timedelta

timeshift_start = (datetime.datetime.strptime('2018-10-05', "%Y-%m-%d") - timedelta(days=3)).strftime("%Y-%m-%d")

In [ ]:
df['SBV_DATE'] = pd.to_datetime(df['SBV_DATE'], format='%Y-%m-%d')

In [ ]:
# extract the date or hour from a datetime

live_df['capping_silo_date'] = live_df['start_hr'].dt.date
live_df['hour'] = live_df['start_hr'].dt.hour

In [ ]:
day_parts_dic = {'Morning' : ('06:00:00', '11:59:59'),
                 'Pre-Peak' : ('12:00:00', '17:24:59'),
                 'Early Peak' : ('17:25:00', '19:59:59'),
                 'Late Peak' : ('20:00:00', '22:59:59'),
                 'Post Peak' : ('23:00:00', '00:29:59'),
                 'Night Time' : ('00:30:00', '05:59:59')}


day_part_lst = []
process_date_range = pd.date_range(date_range[0], date_range[1]) 
for process_day in process_date_range:
    for key, value in day_parts_dic.items():
        start_time = datetime.datetime.strptime(value[0], "%H:%M:%S").time()
        end_time = datetime.datetime.strptime(value[1], "%H:%M:%S").time()
        
        start_datetime = datetime.datetime.combine(process_day, start_time)
        end_datetime = datetime.datetime.combine(process_day, end_time)
        
        if end_datetime < start_datetime:
            end_datetime += datetime.timedelta(days=1)        
        
        day_part_lst.append([key, start_datetime, end_datetime])
        

day_part_df = pd.DataFrame(day_part_lst)

day_part_df

## Text Matching

In [ ]:
from fuzzywuzzy import fuzz

df['match_value'] = df.apply(lambda row: fuzz.ratio(row['gcp_programme_name'], 
                                                    row['ol_programme_name']), axis=1)

df.head(20)

## Charts

Useful blog: https://jakevdp.github.io/PythonDataScienceHandbook/04.10-customizing-ticks.html

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl

today_dt = datetime.datetime.today().strftime("%Y-%m-%d")

fig, ax = plt.subplots()

plt.plot(return_df['SKY_PLUS'], linewidth=2, label='Sky+', color='b')
plt.plot(return_df['SKY_Q'], linewidth=2, label='Sky Q', color='g')
plt.plot(return_df['TOTAL'], linewidth=4, label='All', color='r')
plt.legend(loc='upper left')

fig.suptitle('INC2897270: ROI Account Returns [{}]'.format(today_dt), fontsize=30)
plt.ylabel('Number of Accounts', fontsize=25)
plt.tick_params(labelsize=20)
plt.grid()
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

fig.savefig("INC2897270 ROI Account Returns.png")

In [ ]:
fig, ax = plt.subplots()

today_dt = datetime.datetime.today().strftime("%Y-%m-%d")
plt.suptitle('Number of Accounts Watching BT Sports', fontsize=30)
plt.title('Produced ' + today_dt, fontsize=18)
plt.ylabel('Number of Accounts', fontsize=25)
plt.xlabel('', fontsize=0, color='w')

plt.tick_params(labelsize=20)
ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))

combined_df.plot(x='viewing_date', y='dist_acc', ax=ax, kind='line', label='Daily unique', color='b')
combined_df.plot(x='viewing_date', y='rolling28_count', ax=ax, kind='line', label='28 day rolling', color='g')
combined_df.plot(x='viewing_date', y='rolling56_count', ax=ax, kind='line', label='56 day rolling', color='r')

plt.grid()

today_dt = datetime.datetime.today().strftime("%Y%m%d")
if chart_path == '':
    save_chart = os.getcwd() + '\\'
else:
    save_chart = chart_path + '\\' 

save_chart += 'Ocean BT Sports ' + today_dt + '.png'
fig.savefig(save_chart)

### Complex

In [ ]:
def chart_format_large_numbers(value, tick_number):
    if value >= 10**9:
        return '{:.0f}'.format(value / 10**9)
    elif value >= 10*6:
        return '{:.0f}'.format(value / 10**6)
    elif value >= 10*3:
        return '{:.0f}'.format(value / 10**6)
    elif value < 1:
        return '{:.0%}'.format(value)
    else:
        return '{:.0f}'.format(value / 10**0)
    
    
    


fig, ax = plt.subplots(3, 2, figsize=(17, 14))

xtick_freq = 3

sub_ax = ax[0, 0]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_total_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Total (capped & scaled)')
ol_month_df.plot(y='p11_total_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


sub_ax = ax[0, 1]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_rec_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Recorded (capped & scaled)')
ol_month_df.plot(y='p11_rec_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


sub_ax = ax[1, 0]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_total_cap_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Total (capped NOT scaled)')
ol_month_df.plot(y='p11_total_cap_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


sub_ax = ax[1, 1]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_rec_cap_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Recorded (capped NOT scaled)')
ol_month_df.plot(y='p11_rec_cap_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


sub_ax = ax[2, 0]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_total_uncap_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Total (raw)')
ol_month_df.plot(y='p11_total_uncap_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


sub_ax = ax[2, 1]
sub_ax.tick_params(labelsize=10)
sub_ax.xaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:.0f}'))
sub_ax.yaxis.set_major_formatter(mpl.ticker.StrMethodFormatter('{x:,.0f}'))
ol_month_df.plot(y='p10_rec_uncap_avgmins', ax=sub_ax, kind='line', label='Sky Q', color='b', title='Avg Mins Recorded (raw)')
ol_month_df.plot(y='p11_rec_uncap_avgmins', ax=sub_ax, kind='line', label='Sky +', color='g')
sub_ax.xaxis.set_label_text('', fontsize=0, color='w')
sub_ax.set_xticks(range(len(list(ol_month_df.index.values)))[::xtick_freq])
sub_ax.set_xticklabels(list(ol_month_df.index.values)[::xtick_freq])
#sub_ax.yaxis.set_major_formatter(plt.FuncFormatter(chart_format_large_numbers))
sub_ax.set_ylabel('Minutes', fontsize=10)
sub_ax.grid()


### Spyder inline etc

from: https://stackoverflow.com/questions/29356269/plot-inline-or-a-separate-window-using-matplotlib-in-spyder-ide

In [ ]:
from IPython import get_ipython

# dispplay in console
get_ipython().run_line_magic('matplotlib', 'inline')

# display in seperate window
get_ipython().run_line_magic('matplotlib', 'qt')

## Stats

In [ ]:
from scipy.stats import pearsonr

corr, p_val = pearsonr(df['Deselected Rate'], df['Market Share'])

print('Correlation: {:.3f}; p-value: {:.3f}'.format(corr, p_val))

## Format

In [1]:
my_url='my_url'


print('<a href="{not_my_url}">{}</a>'.format(my_url, my_url, not_my_url='test'))

<a href="test">my_url</a>


## Pandas

### Multiindex pivot

In [ ]:
def multiindex_pivot(df, columns=None, values=None):
    #https://github.com/pandas-dev/pandas/issues/23955
    names = list(df.index.names)
    df = df.reset_index()
    list_index = df[names].values
    tuples_index = [tuple(i) for i in list_index] # hashable
    df = df.assign(tuples_index=tuples_index)
    df = df.pivot(index="tuples_index", columns=columns, values=values)
    tuples_index = df.index  # reduced
    index = pd.MultiIndex.from_tuples(tuples_index, names=names)
    df.index = index
    return df

In [ ]:
def brexitOnly(df):
    brexit_df = df.groupby(by=['mont', 'brexit_status'], as_index=False).sum()
    brexit_df = brexit_df.pivot(index='mont', columns='brexit_status')
    return brexit_df

## Postcodes

In [ ]:
def normalise_postcode(postcode):
    """Return a normalised postcode if valid, or None if not."""
    import re

    NON_ALPHA_RE = re.compile('[^A-Z0-9]+')
    POSTCODE_RE = re.compile('^[A-Z]{1,2}[0-9]{1,2}[A-Z]? [0-9][A-Z]{2}$')
    
    postcode = NON_ALPHA_RE.sub('', postcode.upper())
    postcode = postcode[:-3] + ' ' + postcode[-3:]
    if POSTCODE_RE.match(postcode):
        return postcode
    return None

In [ ]:
def detect_postcode(string):
    """From: https://stackoverflow.com/questions/378157/python-regular-expression-postcode-search"""
    import re
    
    # custom
    method1 = re.findall(r'\b[A-Z]{1,2}[0-9][A-Z0-9]? [0-9][ABD-HJLNP-UW-Z]{2}\b', string)
    
    #regex from #http://en.wikipedia.orgwikiUK_postcodes#Validation                                                                                            
    method2 = re.findall(r'[A-Z]{1,2}[0-9R][0-9A-Z]? [0-9][A-Z]{2}', string)    
    
    return list(set(method1 + method2))